# Examples and exercises for causal models

In [7]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

# Only needed to generate graphs, may be safely ommitted 
# once you comment out relevant cells below
from graphviz import Digraph
from brent import DAG, Query


In [8]:
datadir = Path(os.getcwd()) / 'data'
datadir

WindowsPath('C:/Users/Sabrina Calcina/Documents/GitHub/risk-ai-workshop/notebooks/data')

## Causal model example: hit rate

In [14]:
df = pd.read_csv(datadir / 'hits.csv')
print(df.shape)
df.head()

(5389, 4)


,product_type,days,rating,hit
0,property,3,1,0
1,liability,1,0,0
2,financial,0,1,0
3,liability,3,0,0
4,liability,0,0,1


In [17]:
import os
import graphviz
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz/bin/'
graph_data = "your  graph data"
fie_ext = 'png'
temp_img = 'temp_file'
temp_img_name = "".join([temp_img, '.'+fie_ext])
my_graph= graphviz.Source(graph_data)

dag = DAG(df).add_edge('product_type', 'rating').add_edge('product_type', 'days')\
    .add_edge('rating', 'hit').add_edge('days', 'hit')


dag.plot()

In [23]:
q_given = Query(dag).given(days=1)
dot = q_given.plot()


In [21]:
q_do = Query(dag).do(days=1)
dot = q_do.plot()
dot

In [24]:
# P(H=1 | D=d)
hit_given_d = df.groupby(['days'])
hit_given_d = hit_given_d['hit'].mean()
print('Probability of hit given days=d')
hit_given_d

Probability of hit given days=d


days
0    0.550918
1    0.440580
2    0.288714
3    0.206360
Name: hit, dtype: float64

In [25]:
# P(H=1 | do(D)=d) = \sum P(H=1 | D=d, P=p, R=r) * P(P=p, R=r)
# First term in sum
hit_given_prd = df.groupby(['product_type', 'rating', 'days'])
hit_given_prd = hit_given_prd['hit'].mean()
hit_given_prd

product_type  rating  days
financial     0       0       0.827586
                      1       0.857143
                      2       0.714286
                      3       0.250000
              1       0       0.317136
                      1       0.250000
                      2       0.027778
                      3       0.000000
liability     0       0       0.799784
                      1       0.585915
                      2       0.530686
                      3       0.394040
              1       0       0.296954
                      1       0.200000
                      2       0.041322
                      3       0.031746
property      0       0       0.725000
                      1       0.653846
                      2       0.430894
                      3       0.421659
              1       0       0.358491
                      1       0.186667
                      2       0.045455
                      3       0.016975
Name: hit, dtype: float64

In [26]:
# 2nd sum in P(H=1 | do(D)=1)
pr = df.groupby(['product_type', 'rating'])
p_pr = pr['hit'].count()
p_pr = p_pr / df.shape[0]
p_pr

product_type  rating
financial     0         0.041009
              1         0.163481
liability     0         0.344776
              1         0.149564
property      0         0.120431
              1         0.180739
Name: hit, dtype: float64

In [27]:
# Combine
hit_given_prd = hit_given_prd.reset_index()
p_pr = p_pr.reset_index()
res = pd.merge(hit_given_prd, p_pr, on=['product_type', 'rating'])
res['prob'] = res['hit_x'] * res['hit_y']
res = res.groupby('days')
res = res['prob'].sum()
print('Probability of hit do days=d:')
res

Probability of hit do days=d:


days
0    0.558050
1    0.420425
2    0.283090
3    0.204705
Name: prob, dtype: float64

## Causal models exercise: correlation

Reproduce and try to break the spurious correlation between deaths by poisonous spider bites and the lenghts of winning words in the Scripps national spelling bee.

The fatality data from the CDC can be found here: `notebooks/data/cdc-underlying-cause-of-death-1998-2018.txt`, and the spelling bee data can be found below.

Difficulty: **

(1557, 8)


,Notes,Year,Year Code,Cause of death,Cause of death Code,Deaths,Population,Crude Rate
57,NaN,1999.0,1999.0,Contact with venomous spiders,X21,6.0,279040168.0,Unreliable
135,NaN,2000.0,2000.0,Contact with venomous spiders,X21,5.0,281421906.0,Unreliable
215,NaN,2001.0,2001.0,Contact with venomous spiders,X21,5.0,284968955.0,Unreliable
289,NaN,2002.0,2002.0,Contact with venomous spiders,X21,10.0,287625193.0,Unreliable
364,NaN,2003.0,2003.0,Contact with venomous spiders,X21,8.0,290107933.0,Unreliable


In [32]:
# From https://en.wiktionary.org/wiki/Appendix:Scripps_winning_words
scripps_winners_raw = '''
    1925: gladiolus
    1926: abrogate
    1927: luxuriance
    1928: albumen
    1929: asceticism
    1930: fracas
    1931: foulard
    1932: knack
    1933: torsion
    1934: deteriorating
    1935: intelligible
    1936: interning
    1937: promiscuous
    1938: sanitarium
    1939: canonical
    1940: therapy
    1941: initials
    1942: sacrilegious

The Bee was suspended during the WWII years of 1943–1945.

    1946: semaphore
    1947: chlorophyll
    1948: psychiatry
    1949: dulcimer
    1950: meerschaum [1] / meticulosity
    1951: insouciant
    1952: vignette
    1953: soubrette
    1954: transept
    1955: crustaceology
    1956: condominium
    1957: n/a [2]
    1958: syllepsis
    1959: catamaran
    1960: eudaemonic
    1961: smaragdine
    1962: n/a [3]
    1963: equipage
    1964: sycophant
    1965: eczema
    1966: ratoon
    1967: chihuahua
    1968: abalone
    1969: interlocutory
    1970: croissant
    1971: shalloon
    1972: macerate

    1973: vouchsafe
    1974: hydrophyte
    1975: incisor
    1976: narcolepsy
    1977: cambist
    1978: deification
    1979: maculature
    1980: elucubrate
    1981: sarcophagus
    1982: psoriasis
    1983: Purim
    1984: luge
    1985: milieu
    1986: odontalgia
    1987: staphylococci
    1988: elegiacal
    1989: spoliator
    1990: fibranne
    1991: antipyretic
    1992: lyceum
    1993: kamikaze
    1994: antediluvian
    1995: xanthosis
    1996: vivisepulture
    1997: euonym
    1998: chiaroscurist
    1999: logorrhea
    2000: demarche
    2001: succedaneum
    2002: prospicience
    2003: pococurante
    2004: autochthonous
    2005: appoggiatura
    2006: Ursprache
    2007: serrefine
    2008: guerdon
    2009: Laodicean
    2010: stromuhr
    2011: cymotrichous
    2012: guetapens
    2013: knaidel
    2014: stichomythia / feuilleton
    2015: scherenschnitte / nunatak
    2016: Feldenkrais / gesellschaft
    2017: marocain
    2018: koinonia
    2019: auslaut / erysipelas / bougainvillea [4] / aiguillette / pendeloque / palama / cernuous / odylic
'''




In [47]:
data = pd.read_csv( datadir / "cdc-underlying-cause-of-death-1998-2018.txt", sep = "\t")
print(data.shape)
data.head()

data["Cause of death"].unique()
a = data[data["Cause of death"] == 'Contact with venomous spiders']
a.head()


(1557, 8)


,Notes,Year,Year Code,Cause of death,Cause of death Code,Deaths,Population,Crude Rate
57,NaN,1999.0,1999.0,Contact with venomous spiders,X21,6.0,279040168.0,Unreliable
135,NaN,2000.0,2000.0,Contact with venomous spiders,X21,5.0,281421906.0,Unreliable
215,NaN,2001.0,2001.0,Contact with venomous spiders,X21,5.0,284968955.0,Unreliable
289,NaN,2002.0,2002.0,Contact with venomous spiders,X21,10.0,287625193.0,Unreliable
364,NaN,2003.0,2003.0,Contact with venomous spiders,X21,8.0,290107933.0,Unreliable


In [57]:
data2 = scripps_winners_raw.split("\n")

sez = []
for element in data2:
    element.replace(" ", "")
    element.replace("The Bee was suspended during the WWII years of 1943–1945.", "")
    sez.append(element.split(":"))

data22 = pd.DataFrame(sez)    
    
print(data22)

    
    


           0                                                  1
0                                                          None
1       1925                                          gladiolus
2       1926                                           abrogate
3       1927                                         luxuriance
4       1928                                            albumen
..       ...                                                ...
93      2016                         Feldenkrais / gesellschaft
94      2017                                           marocain
95      2018                                           koinonia
96      2019   auslaut / erysipelas / bougainvillea [4] / ai...
97                                                         None

[98 rows x 2 columns]


## Causal models exercise: do-calculus

As before, take K to be your Karma, H to be the hours you spend in the gym lifting weight, and then W be the weight you can bench press. 

You are the parent of a very young child, so Karma will punish you for devoting too much time to your triceps and neglecting your partner and baby. Let $G$ be this causal graph, as shown below.

In [30]:
dot = Digraph(engine='neato')
dot.attr('node')
dot.node('K')
dot.node('H')
dot.node('W')

dot.edge('K', 'H')
dot.edge('K', 'W')
dot.edge('H', 'W')

#dot

1. Draw the graphs $G_\underline{W}$ and $G_\overline{H}$. Difficulty: *A
2. Write out formulas for $P(W=1 | H=1)$ and $P(W=1|\, \mathrm{do}(H) = 1)$. Difficulty: **

3. Calculate $P(W=1 | H=1)$ and $P(W=1|\, \mathrm{do}(H) = 1)$ for a Bayesian network fitted to the sample data from $(K, H, W)$ in `notebooks/data/karma_weights.csv`. Hint: the `Query` class of [https://koaning.github.io/brent/](https://koaning.github.io/brent/) can be used. Interpret the results in a qualitative way, i.e. how do you think Karma should work in this situation? Difficulty: **

In [ ]:
#G_W
dotw = Digraph(engine='neato')
dotw.attr('node')
dotw.node('K')
dotw.node('H')
dotw.node('W')
dotw.edge('K', 'H')

#G_h
doth = Digraph(engine='neato')
doth.attr('node')
doth.node('K')
doth.node('H')
doth.node('W')

doth.edge('K', 'H')
doth.edge('K', 'W')


## Causal models exercise: Causal calculus

Prove in gory detail that the special case of Causal rule 3 holds. Difficulty: *